In [18]:
import pinecone
from tqdm import tqdm
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("PINECONE_API_KEY")



In [24]:
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key=api_key,
    index='haystack-lfqa',
    similarity="cosine",
    embedding_dim=768
)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '7b9009dcbd0c319fcb72093806894f4f', 'Date': 'Sat, 13 Jan 2024 04:15:38 GMT', 'Server': 'Google Frontend', 'Content-Length': '72', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Capacity Reached. Increase your quota or upgrade to create more indexes.


In [22]:
#Initialize pinecone doc store
from haystack.document_stores import PineconeDocumentStore
pinecone.init(      
	api_key=api_key,      
	environment='gcp-starter'      
)      
index = pinecone.Index('haystack-lfqa')

document_store = PineconeDocumentStore(index=index,api_key=api_key)

AttributeError: 'Index' object has no attribute 'replace'

In [ ]:
#Textbook Extraction to make digestable for docu store
from TextBookExctraction import Process_PDF
#max_chunk_length is the max token length of each vector within the database
#stride refers to the step taken to find the middle of each vector. 
#If stride is 2 and if max_length is 3, we move 2 steps forwards and each vector will contain 3 tokens with an overlap of 1
# [1,2,3] , [3,4,5], [5,6,7], ... , [n-1,n,n+1]            with each array referring to a chunk/vector
pdf_processor = Process_PDF(pdf_path="./Textbooks/CrackingTheCodingInterview.pdf")
text = pdf_processor.extract_text_from_pdf()
cleaned_text = pdf_processor.preprocess_text(text)
text_chunks = pdf_processor.segment_text(cleaned_text, max_chunk_length=500, stride=200)
print(len(text_chunks))

In [ ]:
import torch
#Initialize retriever model
from haystack.nodes import EmbeddingRetriever
retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)


In [ ]:
batch_size = 256
total_doc_count = len(text_chunks)

counter = 0
docs = []
for d in text_chunks:
    docs.append(d)
    counter += 1
    if counter % batch_size == 0 or counter == total_doc_count:
        # Index documents to Pinecone
        document_store.write_documents(docs)
        docs.clear()
